# Análise de Dados Agrícolas

Este projeto analisa dados de produtos agrícolas para compreender os impactos do solo e clima nos tipos de culturas.

**Objetivos:**
- Realizar Análise Exploratória de Dados (AED)
- Conduzir análise descritiva
- Definir perfis ideais de solo/clima
- Construir e avaliar modelos preditivos

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from scipy.stats import f_oneway

# Definir estilo dos gráficos
plt.style.use('seaborn-darkgrid')
sns.set_context("notebook", font_scale=1.2)

## Carregamento e Exploração Inicial dos Dados

In [ ]:
# Carregar o conjunto de dados
df = pd.read_csv('Atividade_Cap_14_produtos_agricolas.csv')

# Inspeção inicial
print("Primeiras 5 linhas:")
display(df.head())

print("\nInformações do Dataset:")
display(df.info())

print("\nEstatísticas Descritivas:")
display(df.describe())

## Análise Exploratória de Dados (AED)

In [ ]:
# Verificar e tratar valores ausentes
print("Valores Ausentes:")
display(df.isnull().sum())
df = df.fillna(df.mean(numeric_only=True))

# Criar histogramas
plt.figure(figsize=(12, 8))
df.hist(bins=20, figsize=(12, 8), color='skyblue', edgecolor='black')
plt.tight_layout()
plt.savefig('eda_histogramas.png')
plt.show()

# Matriz de correlação
plt.figure(figsize=(10, 8))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='viridis', fmt='.2f', linewidths=0.5)
plt.title('Matriz de Correlação das Características')
plt.savefig('eda_correlacao.png')
plt.show()

## Análise Descritiva com Visualizações

In [ ]:
# Temperatura vs. Umidade
plt.figure(figsize=(12, 6))
sns.scatterplot(x='temperature', y='humidity', hue='label', data=df, palette='tab20', s=100)
plt.title('Temperatura vs. Umidade por Tipo de Cultura')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Umidade (%)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('desc_temp_umidade.png')
plt.show()

# Análise NPK
plt.figure(figsize=(12, 6))
df.groupby('label')[['N', 'P', 'K']].mean().plot(kind='bar', color=['#FF9999', '#66B2FF', '#99FF99'])
plt.title('Níveis Médios de N, P, K por Tipo de Cultura')
plt.xlabel('Tipo de Cultura')
plt.ylabel('Nível Médio de Nutrientes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('desc_npk.png')
plt.show()

# Visualizações adicionais
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

sns.boxplot(x='label', y='ph', data=df, ax=ax1)
ax1.set_title('Distribuição do pH por Cultura')
ax1.tick_params(axis='x', rotation=45)

sns.violinplot(x='label', y='rainfall', data=df, ax=ax2)
ax2.set_title('Distribuição da Precipitação por Cultura')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('desc_distribuicoes.png')
plt.show()

## Análise do Perfil Ideal de Solo/Clima

In [ ]:
# Calcular perfis
profile = df.groupby('label').mean(numeric_only=True)
ideal_profile = df.mean(numeric_only=True)

print("Perfil Médio por Cultura:")
display(profile)

print("\nPerfil Ideal (Média Global):")
display(ideal_profile)

# Teste ANOVA
groups = [group['temperature'].values for name, group in df.groupby('label')]
f_stat, p_val = f_oneway(*groups)
print(f"\nTeste ANOVA para Temperatura: Estatística-F={f_stat:.2f}, valor-p={p_val:.4f}")

## Modelagem Preditiva

In [ ]:
# Preparação dos dados
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])
X = df.drop(['label', 'label_encoded'], axis=1)
y = df['label_encoded']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Definir e treinar modelos
models = {
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'Floresta Aleatória': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Rede Neural': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    print(f"\nDesempenho do {name}:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    
    # Matriz de confusão
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(f'Matriz de Confusão - {name}')
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.savefig(f'modelo_mc_{name.lower().replace(" ", "_")}.png')
    plt.show()

## Conclusão

Principais descobertas:
1. Diferentes culturas mostram requisitos distintos para nutrientes do solo e condições climáticas
2. O classificador Floresta Aleatória alcançou o melhor desempenho preditivo
3. Temperatura e umidade são fortes indicadores para adequação da cultura

Limitações e Trabalhos Futuros:
- Considerar dados temporais para efeitos sazonais
- Incorporar fatores ambientais adicionais
- Explorar oportunidades de engenharia de características